In [25]:
#이미지 업로드
#파일 이름 변경할 것

from google.colab import files

uploaded = files.upload() 
for name, data in uploaded.items():
  
  with open(name, 'wb') as f:
    f.write(data)
    print ('saved file', name)

Saving style_1.jpg to style_1.jpg
saved file style_1.jpg


In [1]:
from keras.applications import inception_v3
from keras import backend as K

K.set_learning_phase(0)   # 이 명령어는 모든 훈련 연산을 비활성화 함
model = inception_v3.InceptionV3(weights = 'imagenet', include_top = False)

Using TensorFlow backend.


87916544/87910968 [==============================] - 1s 0us/step


In [3]:
model.summary()

Model: "inception_v3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None, None, 3 0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, None, None, 3 864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, None, None, 3 96          conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, None, None, 3 0           batch_normalization_1[0][0]      
_______________________________________________________________________________________

In [2]:
#딥드림 설정하기
layer_contributions = {
    'mixed2' : 0.2,
    'mixed3' : 3.,
    'mixed4' : 2.,
    'mixed5' : 1.5
}

In [5]:
#최대화할 손실 정의하기
layer_dict = dict([(layer.name, layer) for layer in model.layers])

loss = K.variable(0.)
for layer_name in layer_contributions:
  coeff = layer_contributions[layer_name]
  activation = layer_dict[layer_name].output

  scaling = K.prod(K.cast(K.shape(activation), 'float32'))
  loss = loss + coeff * K.sum(K.square(activation[:, 2: -2, 2: -2, :])) / scaling

In [22]:
 #이 텐서는 생성된 딥드림 이미지를 저장합니다
dream = model.input

# 손실에 대한 딥드림 이미지의 그래디언트를 계산합니다
grads = K.gradients(loss, dream)[0]

# 그래디언트를 정규화합니다(이 기교가 중요합니다)
grads /= K.maximum(K.mean(K.abs(grads)), 1e-7)

# 주어진 입력 이미지에서 손실과 그래디언트 값을 계산할 케라스 Function 객체를 만듭니다
outputs = [loss, grads]
fetch_loss_and_grads = K.function([dream], outputs)

def eval_loss_and_grads(x):
    outs = fetch_loss_and_grads([x])
    loss_value = outs[0]
    grad_values = outs[1]
    return loss_value, grad_values

# 이 함수는 경사 상승법을 여러 번 반복하여 수행합니다
def gradient_ascent(x, iterations, step, max_loss=None):
    for i in range(iterations):
        loss_value, grad_values = eval_loss_and_grads(x)
        if max_loss is not None and loss_value > max_loss:
            break
        print('...', i, '번째 손실 :', loss_value)
        x += step * grad_values
    return x

In [23]:
import scipy
from keras.preprocessing import image

def resize_img(img, size):
    img = np.copy(img)
    factors = (1,
               float(size[0]) / img.shape[1],
               float(size[1]) / img.shape[2],
               1)
    return scipy.ndimage.zoom(img, factors, order=1)


def save_img(img, fname):
    pil_img = deprocess_image(np.copy(img))
    image.save_img(fname, pil_img)


def preprocess_image(image_path):
    # 사진을 열고 크기를 줄이고 인셉션 V3가 인식하는 텐서 포맷으로 변환하는 유틸리티 함수
    img = image.load_img(image_path)
    img = image.img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = inception_v3.preprocess_input(img)
    return img


def deprocess_image(x):
    # 넘파이 배열을 적절한 이미지 포맷으로 변환하는 유틸리티 함수
    if K.image_data_format() == 'channels_first':
        x = x.reshape((3, x.shape[2], x.shape[3]))
        x = x.transpose((1, 2, 0))
    else:
        # inception_v3.preprocess_input 함수에서 수행한 전처리 과정을 복원합니다
        x = x.reshape((x.shape[1], x.shape[2], 3))
    x /= 2.
    x += 0.5
    x *= 255.
    x = np.clip(x, 0, 255).astype('uint8')
    return x

In [27]:
import numpy as np

# 하이퍼파라미터를 바꾸면 새로운 효과가 만들어집니다
step = 0.01  # 경상 상승법 단계 크기
num_octave = 3  # 경사 상승법을 실행할 스케일 단계 횟수
octave_scale = 1.4  # 스케일 간의 크기 비율
iterations = 20  # 스케일 단계마다 수행할 경사 상승법 횟수

# 손실이 10보다 커지면 이상한 그림이 되는 것을 피하기 위해 경사 상승법 과정을 중지합니다
max_loss = 10.

# 사용할 이미지 경로를 씁니다
base_image_path = 'style_1.jpg'

# 기본 이미지를 넘파이 배열로 로드합니다
img = preprocess_image(base_image_path)

# 경사 상승법을 실행할 스케일 크기를 정의한 튜플의 리스트를 준비합니다
original_shape = img.shape[1:3]
successive_shapes = [original_shape]
for i in range(1, num_octave):
    shape = tuple([int(dim / (octave_scale ** i)) for dim in original_shape])
    successive_shapes.append(shape)

# 이 리스트를 크기 순으로 뒤집습니다
successive_shapes = successive_shapes[::-1]

# 이미지의 넘파이 배열을 가장 작은 스케일로 변경합니다
original_img = np.copy(img)
shrunk_original_img = resize_img(img, successive_shapes[0])

for shape in successive_shapes:
    print('처리할 이미지 크기', shape)
    img = resize_img(img, shape)
    img = gradient_ascent(img,
                          iterations=iterations,
                          step=step,
                          max_loss=max_loss)
    upscaled_shrunk_original_img = resize_img(shrunk_original_img, shape)
    same_size_original = resize_img(original_img, shape)
    lost_detail = same_size_original - upscaled_shrunk_original_img

    img += lost_detail
    shrunk_original_img = resize_img(original_img, shape)
    save_img(img, fname='dream_at_scale_' + str(shape) + '.png')

save_img(img, fname='final_dream.png')


처리할 이미지 크기 (98, 132)
... 0 번째 손실 : 0.050332975
... 1 번째 손실 : 0.054724306
... 2 번째 손실 : 0.06821226
... 3 번째 손실 : 0.08120372
... 4 번째 손실 : 0.0945044
... 5 번째 손실 : 0.10785038
... 6 번째 손실 : 0.119424984
... 7 번째 손실 : 0.13186514
... 8 번째 손실 : 0.14242665
... 9 번째 손실 : 0.15208453
... 10 번째 손실 : 0.1651834
... 11 번째 손실 : 0.17526507
... 12 번째 손실 : 0.18473795
... 13 번째 손실 : 0.19503042
... 14 번째 손실 : 0.2028316
... 15 번째 손실 : 0.21102086
... 16 번째 손실 : 0.2222934
... 17 번째 손실 : 0.23211606
... 18 번째 손실 : 0.24039978
... 19 번째 손실 : 0.24668293
처리할 이미지 크기 (138, 185)
... 0 번째 손실 : 1.0073044
... 1 번째 손실 : 1.5279639
... 2 번째 손실 : 2.1076317
... 3 번째 손실 : 2.5612152
... 4 번째 손실 : 3.034788
... 5 번째 손실 : 3.5412936
... 6 번째 손실 : 3.9964163
... 7 번째 손실 : 4.3430743
... 8 번째 손실 : 4.787435
... 9 번째 손실 : 5.1056395
... 10 번째 손실 : 5.34088
... 11 번째 손실 : 5.748835
... 12 번째 손실 : 6.009975
... 13 번째 손실 : 6.4528327
... 14 번째 손실 : 6.7412086
... 15 번째 손실 : 7.1121645
... 16 번째 손실 : 7.31466
... 17 번째 손실 : 7.789164
... 18 번째 손실 : 8.